Here is the full model for Google Colab. It classifies the dog breed on an image, then based off the answer it passes that dog breed into the language model to output a naturally spoken fact about said breed. The image classification model should run on any CPU, but the language model requires a CUDA GPU. The code below also records the softmax values of the model's top 3 and the answer the user was expecting in a CSV file.

<font color='red'>First you need to install the dependencies for the language model. You can do so by running the cell bellow. If you do not have a CUDA GPU, you can skip this cell. Given that you are most likely doing this in Google Colab, you can access a GPU by clicking on the triangle pointing downward under "Comment" in the top left corner, then going to "Change Runtime Type" and choosing the "T4 GPU".

In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install xformers trl peft accelerate triton bitsandbytes #keep an eye out on the xformers version. Usually you want one version before the latest; causes errors often

You will need LoRA adapters and a pytorch dictionary file (.pth) for image calssification. The cell below downloads them from a Google Drive. The last four lines download a couple of images to test around as well.

<font color='red'>Run the cell to get the adapters and pytorch dictionary. If you have your own images of a dog instead feel free to erase the last four lines here.

In [2]:
import gdown

#Download the folder containg the LoRA adapters
url = "https://drive.google.com/drive/folders/1cf-XTMMZb42k5bs3AR3xMRs9V5koXZnQ"
gdown.download_folder(url, quiet=True, use_cookies=False)

#Download the .pth image model
url = 'https://drive.google.com/file/d/1ABhX5RLxh_-OSubYhwV7RhgqeeaHYOpl/view?usp=sharing'
output_path = 'DogImageModel.pth'
gdown.download(url, output_path, quiet=False,fuzzy=True)

#Download the Dog_List.txt
url = 'https://drive.google.com/file/d/13Va0eqByu_CiR1O3xXO-OZLfJjY8NwwM/view?usp=sharing'
output_path = 'Dog_List.txt'
gdown.download(url, output_path, quiet=False,fuzzy=True)

#Download some sample images to try the model on
gdown.download('https://drive.google.com/file/d/1PLTSwWyPoMwgxBU9gTwDoYSzokqAJdVF/view?usp=sharing', 'Dogpic1.jpg', quiet=False,fuzzy=True)
gdown.download('https://drive.google.com/file/d/1LDp6ru_AhUQd4pHecIBMqyiw2tQdKpfe/view?usp=sharing', 'Dogpic2.jpg', quiet=False,fuzzy=True)
gdown.download('https://drive.google.com/file/d/1uA2HJknMZyeR8X3vVvJgTq8pJAmCzaCp/view?usp=sharing', 'Dogpic3.jpg', quiet=False,fuzzy=True)
gdown.download('https://drive.google.com/file/d/1kY5cxn7bTtNm2waDn9hPxg6E5rC7UZYS/view?usp=sharing', 'Dogpic4.jpg', quiet=False,fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1ABhX5RLxh_-OSubYhwV7RhgqeeaHYOpl
From (redirected): https://drive.google.com/uc?id=1ABhX5RLxh_-OSubYhwV7RhgqeeaHYOpl&confirm=t&uuid=8fb5c8c6-1d76-4e2d-87a7-87efb6e822bf
To: /content/DogImageModel.pth
100%|██████████| 44.9M/44.9M [00:00<00:00, 58.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=13Va0eqByu_CiR1O3xXO-OZLfJjY8NwwM
To: /content/Dog_List.txt
100%|██████████| 1.10k/1.10k [00:00<00:00, 2.13MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PLTSwWyPoMwgxBU9gTwDoYSzokqAJdVF
To: /content/Dogpic1.jpg
100%|██████████| 155k/155k [00:00<00:00, 70.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1LDp6ru_AhUQd4pHecIBMqyiw2tQdKpfe
To: /content/Dogpic2.jpg
100%|██████████| 41.8k/41.8k [00:00<00:00, 64.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1uA2HJknMZyeR8X3vVvJgTq8pJAmCzaCp
To: /content/Dogpic3.jpg
100%|██████████| 80.8k/80.8k [00:00<00:00, 43.7MB/s]
Downloading...
From: https:/

'Dogpic4.jpg'

Down below is the script to load and run the dog classification model. If a CUDA GPU model is available, it will also load the language model here. It is device agnostic so it will automatically detect CUDA.

<font color='red'>Focusing on the image_path variable, feel free  to use one of the images provided by the download above (Dogpic1.jpg - Dogpic.jpg4). You can also use your own photo of a dog from the selected dog breed list. If you choose to use your own photo, make sure to upload it and adjust the image_path to wherever your photo is saved in the Colab files (Google Colab ownly allows temporary uploads so it will be erased at the end of your runtime). You might need to change the extension to .jpg before uploading. After you have uploaded and adjusted the image_path variable appropriately, you can run the cell.

In [3]:
#Specify the path to your image here in the Google Colab notebook. You can copy the path by right clicking on the file you want to use. Usually an uploaded image is found in the /content folder
image_path = '/content/Dogpic1.jpg'
early_stop = False #The scripts allows for manual input of a dog breed if the breed isn't in the top three. "early_stop" controls whether you want to allow the user to run the LLM after manual input.
                   #True stops the code from runing the LLM for a fun fact; False allows the user to get a random fat based off the manually inputed dog breed

import torch
from PIL import Image
from torchvision.transforms import v2
import torchvision.models as models
import os

# Load the pre-trained model that was used in the training script. In this case it was ResNet18 model
model = models.resnet18()

# Modify the final fully connected layer to have 73 output classes, same as in the training script
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 73)

# Directory to load the .pth file that was acreated by the training script
model.load_state_dict(torch.load('/content/DogImageModel.pth', map_location=torch.device('cpu')))

# Set the model to evaluation mode
model.eval()

# Automatically detect the available device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

#Open a specified image
image = Image.open(image_path)

#Open the first file in a folder directory with the specified filetypes
"""image = Image.open(next((f for f in os.listdir('/content/') if f.endswith(('.jpg', '.jpeg', '.png', '.webp', 'bmp'))), None))"""

#Transforms the images to how they were tested for the model to read for inference. Keep Exactly the same as the transformation for the test and valid sets. No randomizing here!
transforms_test = v2.Compose([
    v2.Resize((224, 224), antialias=True),
    v2.CenterCrop((224, 224)),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

#Apply the transformation to your image
transformed_img = transforms_test(image)

# Add a batch dimension (1, 3, 224, 224)
transformed_img = transformed_img.unsqueeze(0).to(device)  # Move data to device

output = model(transformed_img)

with open('Dog_List.txt', 'r') as f:
    labels = [line.strip() for line in f.readlines()]

#I have the breed_nicknames dictionary set because some breeds arent recognized that much by the official breed, such as the ones below.
breed_nicknames = {
    'Xoloitzcuintli': ' (Mexican Hairless)',
    'Staffordshire-Bull-Terrier': ' (Pitbull)',
    'Pembroke-Welsh-Corgi': ' (Corgi)',}


# Apply softmax to get probabilities
output_softmax = torch.nn.functional.softmax(output, dim=1)

# Get the top 3 predictions by using pytorch to find the Top-3 K values
topk_values, topk_indices = torch.topk(output_softmax, 3)
topk_indices = topk_indices.tolist()[0]  # Convert tensor to list of integers
topk_labels = [labels[index] for index in topk_indices] # Use the indices to get the labels. This turns the highest three values in the tensor into the labels of the dog breeds

# Print all probabilities. This is useful if you would like to record all of the labels for more detailed data.
"""for i, prob in enumerate(output_softmax.tolist()[0]):
    print(f"{labels[i]}: {prob:.4f}")"""

print("Choose a dog breed:")


#For data feedback, this is a function to save user data. It saves the probabilities of the Top-3, their respective labels, and the label specified by the user
def save_answer(answer, label=None):
    filename = "user_answers.csv"
    if not os.path.exists(filename):
        open(filename, 'w').close()  # Create file if it doesn't exist with the categories as the first row, then closes it
        with open(filename, 'w') as file:
            file.write("Probability 1, Probability 2, Probability 3, Label Rank 1, Label Rank 2, Label Rank 3, User Label, Top-3?\n") #The categories to be written at teh top if the file does not exist

    probabilities = ', '.join(map(str, topk_values.tolist())).strip('[]')
    labels = ', '.join(topk_labels)
    correct_label = label if label else answer
    top3 = 'Y' if int(answer) in range(1, 4) else 'N'  # Check if answer is in top 3

    with open(filename, 'a') as file:  # Append the user results
        file.write(f"{probabilities}, {labels}, {correct_label}, {top3}\n")

#While loop that repeatedly prompts user for dog breed choice until the user provides a valid response.
while True:
    for i, label in enumerate(topk_labels):
        # Check if the breed has a nickname, then print it with the nickname in parantheses
        if label in breed_nicknames:
            print(f"{i+1}. {label}{breed_nicknames[label]}")
        else:
            print(f"{i+1}. {label}")
    print("4. None of these.")
    try:
        choice = int(input("Enter the number of your chosen breed: ")) - 1
        if choice in [0, 1, 2]:
            dog_breed = topk_labels[choice]
            print(f"My dog is a {dog_breed.replace('-', ' ')} breed.")
            save_answer(choice + 1, dog_breed)
            break
        elif choice == 3:  # If the dog breed is not in the top-3, the following instead is ran
            dog_breed = input("Please enter your dog's breed (50 characters or less): ")
            if len(dog_breed) <= 50: #Allows the user to only input 50 characters
                early_stop = early_stop #This blocks the language model from operating when a manual input is inserted. Set to False if you want the language model to work on manual input
                print("Thank you for letting us know! We'll work on improving our model for", dog_breed, "breeds.")
                save_answer(4, dog_breed)
                break
            else:
                print("Sorry, that's too long. Please keep it under 50 characters.")
        else:
            print("Invalid choice. Please enter 1, 2, 3, or 4.")
    except ValueError:
        print("Invalid input. Please enter a number.")



<ipython-input-3-1da93f0817ea>:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/DogImageModel.pth', map_location=torch.device('cp

Choose a dog breed:
1. Saint-Bernard
2. American-Foxhound
3. Beagle
4. None of these.
Enter the number of your chosen breed: 1
My dog is a Saint Bernard breed.


In [4]:

#Load the LoRA adapters and set FastLanguageModel for inference (if a CUDA GPU is present)
#Wont load if the user answered number 4 and you DID NOT change early_stop to True in the while block
if not early_stop:
  if torch.cuda.is_available():
      from unsloth import FastLanguageModel


      max_seq_length = 2048
      dtype = None
      load_in_4bit = True

      model, tokenizer = FastLanguageModel.from_pretrained( #same parameters as it was trained on.
          model_name = "/content/Dog-LoRA", #Directory to the folder (not the file) where your model is saved. Any of the save methods from the Unsloth training should work
          max_seq_length = max_seq_length,
          dtype = dtype,
          load_in_4bit = load_in_4bit,
      )
      FastLanguageModel.for_inference(model)

      #Need to set the prompt again
      alpaca_prompt = """

      ### label:
      {}

      ### text:
      {}"""

      labels = tokenizer(
          [
              alpaca_prompt.format(
                  f"Please tell me something interesting about the {dog_breed} Dog",
                  "",
              )
          ], return_tensors = "pt").to("cuda")


  else:
      print("Language model output is only available for GPU hardware")

try:
    texts = model.generate(**labels, max_new_tokens = 128, use_cache = True)
    print(tokenizer.batch_decode(texts))
except Exception as e:
    pass  # Ignore the error


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.8: Fast Llama patching. Transformers: 4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

Unsloth 2024.12.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


['<|begin_of_text|>\n\n      ### label:\n      Please tell me something interesting about the Saint-Bernard Dog\n\n      ### text:\n       Saint-Bernard: Saint Bernards are known to be good with children of all ages and they are very gentle and tolerant of them. However they are large dogs and they can easily knock a toddler over which is why it is always best to supervise any interactions between toddlers and dogs.<|end_of_text|>']


If you loaded the language model in the previouos cell, the next cell is for running the dog_breed variable through the model. It is in a seperate cell so you may run the output several times without reloading the LLM each time. However, if you run the image classifier again, you must run the cell above as well.

In [6]:
texts = model.generate(**labels, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(texts)

['<|begin_of_text|>\n\n      ### label:\n      Please tell me something interesting about the Saint-Bernard Dog\n\n      ### text:\n       Saint-Bernard: The Saint Bernard is a large dog breed with a long history of helping people. They were originally bred to be working dogs in the Swiss Alps, where they were used to rescue travelers who had become stranded in the mountains.<|end_of_text|>']